**Note: This is my EDA.  It's not pretty or nice and probably has some mistakes along the way.  Check out the associated `.py` file for a linted/formatted version of the solution.**

In [2]:
import datetime
from pathlib import Path
import sys
import math

import polars as pl

# Gross hack to get jupyter to find hod stuff.
sys.path.append(str(Path(Path.cwd().parent)))
from hod.utils import get_data, get_male_names

pl.Config.set_tbl_rows(n=100)

polars.config.Config

# The Collector

(JAMES: Emphasis below mine when I think something will be useful.)

REF: https://hanukkah.bluebird.sh/5784/8/

“Oh that damned woman! She moved in, clogged my bathtub, left her coupons all over the kitchen, and then just vanished one night without leaving so much as a note.

Except she did leave behind that nasty carpet. I spent months cleaning one corner, only to discover a snake hiding in the branches! I knew then that she was never coming back, and I had to get it out of my sight.

“Well, I don’t have any storage here, and it didn’t seem right to sell it, so I gave it to my sister. She wound up getting a newer and more expensive carpet, so she gave it to an acquaintance of hers who collects all sorts of junk. Apparently he owns an entire set of Noah’s collectibles! He probably still has the carpet, even.

“My sister is away for the holidays, but I can have her call you in a few weeks.”

The family dinner is tonight! Can you find the collector’s phone number in time?

In [3]:
data = get_data(data_path=Path('./../data/5784/'))

df_customers = data["customers"]
df_orders = data["orders"]
df_orders_items = data["orders_items"]
df_products = data["products"]

In [39]:
df_collectables = (
    df_products
    .select(["sku", "desc"]).filter(pl.col("sku").str.starts_with("COL"))
    .with_columns(
        pl.col("desc")
        .str.replace("\)", "")
        .str.replace("Noah's ", "")
        .str.split_exact(" (", n=1)
        .alias("split"))
    .unnest("split")
    .rename({"field_0": "item", "field_1": "color"})
    .drop("desc")
)

df_collectables_grouped = (
    df_collectables
    .group_by(["item"])
    .agg("color", "sku")
)

df_customers_with_collectables = (
    df_collectables
    .join(df_orders_items, on="sku")
    .join(df_orders, on="orderid")
    .join(df_customers, on="customerid")
    .select(['customerid', 'phone', 'name', 'sku', 'item', 'color', 'orderid'])
)

df_customers_with_collectables_slim = (
    df_customers_with_collectables
    .select("name", "item", "color", "phone")
)

df_customers_with_collectables_slim.group_by(["name", "phone"]).agg(pl.col("color").count()).sort("color", descending=True)

shape: (8, 3)
┌───────────────┬──────────────────────────────────┬───────────────────────────────────┐
│ item          ┆ color                            ┆ sku                               │
│ ---           ┆ ---                              ┆ ---                               │
│ str           ┆ list[str]                        ┆ list[str]                         │
╞═══════════════╪══════════════════════════════════╪═══════════════════════════════════╡
│ Ark Model     ┆ ["HO Scale"]                     ┆ ["COL0041"]                       │
│ Action Figure ┆ ["blue", "mauve", … "orange"]    ┆ ["COL0166", "COL0483", … "COL934… │
│ Poster        ┆ ["mauve", "red", … "magenta"]    ┆ ["COL0837", "COL2098", … "COL977… │
│ Gift Box      ┆ ["green", "amber", … "orange"]   ┆ ["COL2125", "COL2789", … "COL924… │
│ Lunchbox      ┆ ["amber", "puce", … "blue"]      ┆ ["COL1464", "COL1794", … "COL981… │
│ Bobblehead    ┆ ["blue", "orange", … "white"]    ┆ ["COL0167", "COL0891", … "COL902… │
│ Jerse

name,phone,color
str,str,u32
"""James Smith""","""212-547-3518""",111
"""Tyler Johnson""","""838-701-6325""",37
"""Angel White""","""838-806-1721""",36
"""Elizabeth Reye…","""516-993-9571""",36
"""Hailey Fisher""","""838-838-1754""",34
"""John Martinez""","""631-600-2937""",34
"""Todd Simmons""","""516-374-2476""",34
"""Karen Contrera…","""838-578-7266""",34
"""Brandon Castil…","""914-618-0663""",34
